In [202]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

# 1 Iterators
We will be using an interesting dataset. [link](https://tev.fbk.eu/resources/smartwatch)

From the site:
> The SmartWatch Gestures Dataset has been collected to evaluate several gesture recognition algorithms for interacting with mobile applications using arm gestures. Eight different users performed twenty repetitions of twenty different gestures, for a total of 3200 sequences. Each sequence contains acceleration data from the 3-axis accelerometer of a first generation Sony SmartWatch™, as well as timestamps from the different clock sources available on an Android device. The smartwatch was worn on the user's right wrist. 


In [203]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2025-09-27 14:38:42.898 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /Users/wouter/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 13308.57it/s]


In [204]:
len(train), len(valid)

(81, 20)

In [205]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y

(torch.Size([32, 30, 3]),
 tensor([10,  1, 19,  1,  0, 12, 13,  1, 18,  5,  5,  6, 13, 11, 16,  8, 10, 10,
          1,  6, 19,  5, 10,  6,  1, 18,  6,  1,  5, 13,  9, 11]))

Can you make sense of the shape?
What does it mean that the shapes are sometimes (32, 27, 3), but a second time might look like (32, 30, 3)? In other words, the second (or first, if you insist on starting at 0) dimension changes. Why is that? How does the model handle this? Do you think this is already padded, or still has to be padded?


# 2 Excercises
Lets test a basemodel, and try to improve upon that.

Fill the gestures.gin file with relevant settings for `input_size`, `hidden_size`, `num_layers` and `horizon` (which, in our case, will be the number of classes...)

As a rule of thumbs: start lower than you expect to need!

In [206]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [216]:
# --- Cell #7: Conv1D + RNN (LSTM/GRU) ---
# Vormaanname voor batches uit je dataloader/streamer: X heeft shape (B, T, F)
# Conv1d verwacht (B, C_in, T) -> hier is C_in = F (features), T = tijd.
# Na Conv/Pool transponeren we terug naar (B, T', C_out) en voeren we een RNN uit.

import torch
import torch.nn as nn

# 1) Kies je RNN-type: 'lstm' of 'gru'
rnn_type = "gru"     # zet op "gru" om GRU te testen

# 2) Hyperparameters (goede startwaarden voor >90%)
input_size    = 3
conv_channels = (32, 64)   # twee conv-blokken
kernel_sizes  = (5, 3)     # bijpassende kernels
pool          = 2          # MaxPool1d factor (verkort T)
hidden_size   = 128        # RNN hidden dim na conv
num_layers    = 1          # vaak genoeg na goede conv-front-end
rnn_dropout   = 0.2
bidirectional = True

# 3) Bepaal num_classes robuust
try:
    xb, yb = next(iter(trainstreamer))
    num_classes = int(yb.max().item() + 1)
except Exception:
    num_classes = 10  # fallback; pas aan zodra je labels weet

class CNNRNNClassifier(nn.Module):
    """
    Pipeline:
      In:  x (B, T, F)
      ->   x^T (B, F, T)  # transpose voor Conv1d
      ->   Conv1d/Pool stack: (B, C_out, T')
      ->   transpose terug: (B, T', C_out)
      ->   RNN (LSTM/GRU) over tijd
      ->   laatste (bi)hidden concat -> classificatie-head
    """
    def __init__(self,
                 input_dim,
                 num_classes,
                 rnn_type="lstm",
                 conv_channels=(32, 64),
                 kernel_sizes=(5, 3),
                 pool=2,
                 hidden=128,
                 num_layers=1,
                 rnn_dropout=0.2,
                 bidirectional=True):
        super().__init__()
        assert len(conv_channels) == len(kernel_sizes), "conv_channels en kernel_sizes moeten even lang zijn"

        layers = []
        in_ch = input_dim  # F
        for ch, k in zip(conv_channels, kernel_sizes):
            # Output shape: (B, ch, T) met padding=k//2 ⇒ T blijft ~gelijk, daarna pool ⇒ T' = floor(T / pool)
            layers += [
                nn.Conv1d(in_ch, ch, kernel_size=k, padding=k//2),
                nn.ReLU(),
                nn.MaxPool1d(pool)
            ]
            in_ch = ch
        self.conv = nn.Sequential(*layers)

        self.rnn_type = rnn_type.lower()
        rnn_in = in_ch  # laatste aantal channels uit conv-stack

        if self.rnn_type == "gru":
            self.rnn = nn.GRU(
                input_size=rnn_in,
                hidden_size=hidden,
                num_layers=num_layers,
                batch_first=True,
                dropout=rnn_dropout if num_layers > 1 else 0.0,
                bidirectional=bidirectional
            )
        else:
            self.rnn = nn.LSTM(
                input_size=rnn_in,
                hidden_size=hidden,
                num_layers=num_layers,
                batch_first=True,
                dropout=rnn_dropout if num_layers > 1 else 0.0,
                bidirectional=bidirectional
            )

        out_dim = hidden * (2 if bidirectional else 1)
        self.head = nn.Sequential(nn.Dropout(rnn_dropout), nn.Linear(out_dim, num_classes))

    def forward(self, x):
        # x: (B, T, F)
        x = x.transpose(1, 2)          # -> (B, F, T) voor Conv1d
        z = self.conv(x)               # -> (B, C_out, T')
        z = z.transpose(1, 2)          # -> (B, T', C_out)
        out = self.rnn(z)
        if isinstance(out, tuple):     # LSTM: out = (out, (h, c))
            h = out[1][0]
        else:                          # GRU:  out = (out, h)
            h = out[1]
        # h: (layers*dirs, B, H)
        last = torch.cat([h[-2], h[-1]], dim=-1) if self.rnn.bidirectional else h[-1]
        return self.head(last)

# 4) Instantieer het model in variabele 'model' (je notebook gebruikt 'model' downstream)
model = CNNRNNClassifier(
    input_dim=input_size,
    num_classes=num_classes,
    rnn_type=rnn_type,
    conv_channels=conv_channels,
    kernel_sizes=kernel_sizes,
    pool=pool,
    hidden=hidden_size,
    num_layers=num_layers,
    rnn_dropout=rnn_dropout,
    bidirectional=bidirectional
)

# Handige print over de vormveranderingen:
print(f"[CNNRNN] Conv front-end: channels={conv_channels}, kernels={kernel_sizes}, pool={pool}")
print(f"[CNNRNN] RNN: type={rnn_type.upper()}, hidden={hidden_size}, layers={num_layers}, bidirectional={bidirectional}")
model

[CNNRNN] Conv front-end: channels=(32, 64), kernels=(5, 3), pool=2
[CNNRNN] RNN: type=GRU, hidden=128, layers=1, bidirectional=True


CNNRNNClassifier(
  (conv): Sequential(
    (0): Conv1d(3, 32, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (rnn): GRU(64, 128, batch_first=True, bidirectional=True)
  (head): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=256, out_features=20, bias=True)
  )
)

Test the model. What is the output shape you need? Remember, we are doing classification!

In [214]:
yhat = model(x)
yhat.shape

torch.Size([20])

Test the accuracy

In [217]:
accuracy(y, yhat)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

What do you think of the accuracy? What would you expect from blind guessing?

Check shape of `y` and `yhat`

In [181]:
yhat.shape, y.shape

(torch.Size([32, 20]), torch.Size([32]))

And look at the output of yhat

In [182]:
yhat[0]

tensor([ 0.0615, -0.0759,  0.0669, -0.0312, -0.1881,  0.1662, -0.0910, -0.0354,
        -0.0352,  0.0975, -0.0048, -0.0996,  0.0539, -0.1925, -0.0936, -0.2428,
         0.2182, -0.0504, -0.0415, -0.0028], grad_fn=<SelectBackward0>)

Does this make sense to you? If you are unclear, go back to the classification problem with the MNIST, where we had 10 classes.

We have a classification problem, so we need Cross Entropy Loss.
Remember, [this has a softmax built in](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) 

In [183]:
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(yhat, y)
loss

tensor(2.9956, grad_fn=<NllLossBackward0>)

In [184]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

Using MPS


Set up the settings for the trainer and the different types of logging you want

In [185]:
settings = TrainerSettings(
    epochs=35, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 8, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 35
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 8, 'delta': 0.0}

In [186]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class GRUmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [187]:
config = ModelConfig(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    output_size=20,
    dropout=0.0,
)


In [188]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

with mlflow.start_run():
    mlflow.set_tag("model", "modelname-here")
    mlflow.set_tag("dev", "your-name-here")
    config = ModelConfig(
        input_size=3,
        hidden_size=256,
        num_layers=2,
        output_size=20,
        dropout=0.1,
    )

    model = GRUmodel(
        config=config,
    )

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    if not settings.earlystop_kwargs["save"]:
        tag = datetime.now().strftime("%Y%m%d-%H%M-")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

2025-09-27 14:34:24.623 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20250927-143424
2025-09-27 14:34:24.624 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:01<00:00, 45.37it/s]
2025-09-27 14:34:26.611 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.4045 test 2.0700 metric ['0.2641']
100%|██████████| 81/81 [00:01<00:00, 44.64it/s]
2025-09-27 14:34:28.606 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.5843 test 1.3089 metric ['0.4875']
100%|██████████| 81/81 [00:01<00:00, 47.06it/s]
2025-09-27 14:34:30.495 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.8888 test 0.6660 metric ['0.8219']
100%|██████████| 81/81 [00:01<00:00, 47.00it/s]
2025-09-27 14:34:32.400 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.4376 test 0.2110 metric ['0.9531']
100%|██████████| 81/81 [00:01<00:00, 46.90it/s]
2025-09-27 14:34:34.30

Try to update the code above by changing the hyperparameters.
    
To discern between the changes, also modify the tag mlflow.set_tag("model", "new-tag-here") where you add
a new tag of your choice. This way you can keep the models apart.

In [189]:
trainer.loop() # if you want to pick up training, loop will continue from the last epoch

100%|██████████| 81/81 [00:01<00:00, 47.31it/s]
2025-09-27 14:35:20.733 | INFO     | mltrainer.trainer:report:175 - Resuming epochs from previous training at 57
2025-09-27 14:35:20.755 | INFO     | mltrainer.trainer:report:209 - Epoch 57 train 0.0015 test 0.0064 metric ['0.9969']
2025-09-27 14:35:20.755 | INFO     | mltrainer.trainer:__call__:252 - best loss: 0.0055, current loss 0.0064.Counter 9/8.
2025-09-27 14:35:20.755 | INFO     | mltrainer.trainer:loop:103 - Interrupting loop due to early stopping patience.
2025-09-27 14:35:20.755 | INFO     | mltrainer.trainer:loop:108 - early_stopping_save was false, using latest model.Set to true to retrieve best model.
  0%|          | 0/35 [00:01<?, ?it/s]


In [190]:
mlflow.end_run()